<h2>Week 2 Webscraping</h2>

In [681]:
import requests
import re

<h3> Getting the Page Content </h3>
<p> I first want to get the page content of the website. I will use the requests library to get the page content. </p>
<p> Additionally, I will need to decode the bytes with utf-8. </p>

In [682]:
def get_page_content(url):
  headers = {"User-Agent": "Mozilla/5.0"}
  response = requests.get(url, headers=headers)
  # I am just doing a quick check to make sure the fetch was succesful
  if response.status_code == 200:
    byte_content = response.content
    content = str(byte_content, 'utf-8')
    return content
  else:
    raise Exception("Page retrieval unsuccessful")

<h3> Utilizing Regex to Extract the Desired Data from the Searched Page Contents </h3>
<p> This function is going to use regex to search the contents received from the webpage and save them into a dictionary based on the category. I will also replace the &amp with &. I will convert it into a 2d Array later, I am mostly doing it for bug testing to see if I get the right data.</p>

In [683]:
def parse_metacritic_movie_data():
  movie_data = {'Title':[], 'Release Date':[], 'Score':[], 'Thumbnail':[], 'Description':[]}
  data_keys = list(movie_data.keys())
  content = get_page_content('https://www.metacritic.com/browse/movie/all/all/1998/metascore/?page=2')
  regex_strings = [r'<div data-title="([^"]*)"',r'<div class="c-finderProductCard_meta"><span class="u-text-uppercase">\s+(.*?)\s+</span>',r'data-v-4cdca868>([^<]*)</span>',r'<picture class="c-cmsImage"> <img src="([^"]*)"',r'<div class="c-finderProductCard_description"><span>([^<]*)</span>']
  for i in range(len(regex_strings)):
    movie_data[data_keys[i]]= re.findall(regex_strings[i], content)
    
    for j in range(len(movie_data[data_keys[i]])):
        movie_data[data_keys[i]][j] = movie_data[data_keys[i]][j].replace('&amp;', '&')
        movie_data[data_keys[i]][j] = movie_data[data_keys[i]][j].replace('&quot;', '&')

  return movie_data

<h3> Converting into a 2-Dimensional Array </h3>
<p> The assignment wants this to be in an array so I will oragnize the data into a 2d list or a list of lists. I could have done this earlier but I think that a dictionary was cleaner and allowed me to debug easier.</p>
<p> I have went way too overboard with this it took much longer than I anticipated but it was a self challenge. The most difficult part was formatting with wordwrapping in mind.</p>

In [684]:
# This function takes in a dictionary and converts it into a 2D array for use in the table I am going to print.
def convert_into_2d_array(movie_data): 
  cols = list(movie_data.keys())
  rows = list(zip(*movie_data.values()))
  return [cols] + [list(row) for row in rows]

<h3> Make a Nicely Printed Table </h3>
<p> This table is made with some alternate characters for the border and uses the max length of the column to make sure it stays consistent. I wanted it to look a little boujie.</p>
<p> First I need to create a splitting function for the descriptions because if they are too long I do not want it to scroll forever. I will split the function by 50 characters unless it would in the center of a word, in which case it will go to the end of the word. </p>

In [685]:
# This function takes in a 2D array and prints it in a table format 
def split_text(text, max_width):
    words = text.split()
    lines = []
    current_line = ""

    for word in words:
        # Check if adding the next word exceeds max_width
        if len(current_line) + len(word) + 1 > max_width:
            # If the current line is a very long word
            if not current_line:
                current_line = word
            # Add current_line to lines and reset current_line
            lines.append(current_line)
            current_line = word if current_line else "" 
        else:
            # Add the word to the current line
            current_line += (" " if current_line else "") + word
    
    # Add the last line if it exists
    if current_line:
        lines.append(current_line)
    
    return lines

In [687]:
#overly complicated print ascii table
def print_ascii_table(movie_data):
    if not movie_data:
        print("No movie_data.")
        return

    # Calculate initial column widths based on the longest element
    col_widths = [max(len(str(element)) for element in column) for column in zip(*movie_data)]

    # Index of the thumbnail and description columns
    thumbnail_column_index = 3
    description_column_index = 4

    # Set the width for the thumbnail column to the length of the longest URL
    col_widths[thumbnail_column_index] = max(len(row[thumbnail_column_index]) for row in movie_data)

    # Ensure the description column width is correctly determined
    max_description_length = max([len(line) for row in movie_data 
                                  for line in split_text(row[description_column_index], 50)])
    col_widths[description_column_index] = max_description_length

    # Create table separators with appropriate column widths
    first_separator = '╔' + '╦'.join('═' * (width + 2) for width in col_widths) + '╗'
    inner_separator = '╠' + '╬'.join('═' * (width + 2) for width in col_widths) + '╣'
    last_separator = '╚' + '╩'.join('═' * (width + 2) for width in col_widths) + '╝'

    # Print the table header
    print(first_separator)
    header_row = '║ ' + ' ║ '.join(str(title).center(col_widths[i]) for i, title in enumerate(movie_data[0])) + ' ║'
    print(header_row)
    print(inner_separator)

    # Iterate over each row of movie data
    for row_index, row in enumerate(movie_data[1:], start=1):
        description_lines = split_text(row[description_column_index], col_widths[description_column_index])

        # Print the first line of the row ensuring the description does not exceed its column width
        row_str = '║ ' + ' ║ '.join(str(element).ljust(col_widths[i]) for i, element in enumerate(row[:description_column_index]))
        row_str += ' ║ ' + description_lines[0].ljust(col_widths[description_column_index]) + ' ║'
        print(row_str)
        # Print additional lines for longer descriptions
        for additional_line in description_lines[1:]:
            # Generate padding for the columns preceding the description
            padding = ' ║ '.join(' ' * col_widths[i] for i in range(description_column_index))
            # Adjust padding to account for the left border and space between columns
            padding = '║ ' + padding + (' ' * (-3 + (description_column_index))) + '║ '
            print(padding + additional_line.ljust(col_widths[description_column_index]) + ' ║')

        # Print the separator after each row or the bottom border after the last row
        print(inner_separator if row_index < len(movie_data) - 1 else last_separator)

In [688]:
print_ascii_table(convert_into_2d_array(parse_metacritic_movie_data()))

╔══════════════════════════╦══════════════╦═══════╦═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╦════════════════════════════════════════════════════╗
║          Title           ║ Release Date ║ Score ║                                                                                       Thumbnail                                                                                       ║                    Description                     ║
╠══════════════════════════╬══════════════╬═══════╬═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╬════════════════════════════════════════════════════╣
║ Bulworth                 ║ May 19, 1998 ║ 75    ║ https://www.metacritic.com/a/img/resize/7da83ae00a887a57ac5660b560fab5d3505bb124/